# Week 10: Convolutional Neural Networks

<hr style="border:2px solid gray">

# Outline

<hr style="border:2px solid gray">

# Section One

<hr style="border:2px solid gray">

# Section Two

<hr style="border:2px solid gray">

# Section Three

<hr style="border:2px solid gray">


# Section Four: Exercises